# Feature Engineering

In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sb
import itertools 

In [ ]:
notebook_dir = os.path.dirname(os.path.abspath('__file__'))

data_path = os.path.join(notebook_dir, '../data/raw/car_prices.csv')

df = pd.read_csv(data_path)

## Duplication Removal

In [ ]:
# Some values are not unique because of capital letters
# We will move all to title format

Object_col = df.select_dtypes(include = "object").columns.to_list()
Numerical_col = df.select_dtypes(exclude = "object").columns.to_list()

Number_of_unique_before = df.nunique()
for column in Object_col:
    df[column] = df[column].str.title()
Number_of_unique_after = df.nunique()
print('Reduce of unique values')
pd.DataFrame(Number_of_unique_before - Number_of_unique_after).T

### Make

In [ ]:
pd.unique(df['make'])

In [ ]:
# As we can see, it can be more generalized
print(f'Make number unique before: {df.make.nunique()}')

df['make'] = df['make'].replace({'Bmw': 'BMW', 'Gmc': 'GMC', 'Ram': 'RAM', 'Gmc Truck': 'GMC', 'Dodge Tk': 'Dodge',
       'Mazda Tk': 'Mazda', 'Hyundai Tk': 'Hyundai' , 'Mercedes-B': 'Mercedes',  'Mercedes-Benz': 'Mercedes' , 'Vw': 'VW',
       'Chev Truck': 'Chevrolet', 'Ford Tk': 'Ford' , 'Ford Truck' :'Ford'})
pd.unique(df['make'])

print(f'Make number unique after: {df.make.nunique()}')
pd.unique(df['make'])

In [ ]:
# Check sales by make
make = df['make'].value_counts().index
plt.figure(figsize = (20,10))
plt.title('count data by make', loc = 'center')
sb.countplot(data = df, y = 'make', order = make, width = 1.1)
plt.xlabel('number of cars', loc = 'center')

In [ ]:
# We will move Plymouth and below to "Other" category since there are few observations
print(df[df['make'] == 'Bentley'].shape[0])
print(df[df['make'] == 'Plymouth'].shape[0])

In [ ]:
# We create a function to change to Other category
def group_rare_categories(df, column_name, min_count):
    value_counts = df[column_name].value_counts()
    rare_categories = value_counts[value_counts < min_count].index
    df[column_name] = df[column_name].apply(lambda x: 'other' if x in rare_categories else x)
    return df

In [ ]:
# Anything with less than 28 observations will be moved to Other
print(f"groups that have less than 28 observations before: {df['make'].value_counts()[df['make'].value_counts()<28]}")

df = group_rare_categories(df, 'make', 28)

print(f"groups that have less than 10 observations after: {df['make'].value_counts()[df['make'].value_counts()<28]}")

### Trim

In [ ]:
df['trim'].nunique()

In [ ]:
# Too many trim values, lots of them with <100 observations
# Anything with less than 100 observations will be moved to Other
print(f"groups that have less than 100 observations before: {df['trim'].value_counts()[df['trim'].value_counts()<101]}")

df = group_rare_categories(df, 'trim', 101)

print(f"groups that have less than 100 observations after: {df['trim'].value_counts()[df['trim'].value_counts()<101]}")

### Car Age
With SaleDate we can get Car Age (which is more important for a vehicle sale predictor

In [ ]:
# We check how the data is stored
saledate = df['saledate'].str.split(expand = True)
saledate.head(1)

In [ ]:
# Car sales range
print(f'month data: {pd.unique(saledate.iloc[:,1])}')
print(f'year data: {pd.unique(saledate.iloc[:,3])}')

In [ ]:
# We add a new column salesyear to calculate the age of the vehicle in the future
df = pd.merge(left = df, right = saledate.iloc[:, 3], left_index = True, right_index = True)
df.rename(columns = {3: 'salesyear'}, inplace = True)

In [ ]:
# Number of sales by make
df_sale = df.groupby('salesyear').agg({'salesyear' : 'count'})
plt.figure(figsize = (5,5))
plt.title('cars sales by year', loc = 'center')
plt.pie(df_sale['salesyear'], labels = df_sale['salesyear'] )
plt.legend(labels = df_sale.index, fontsize = 10, loc='upper left')

In [ ]:
# Fill nan with 2015
print(f'num nan sales year before: {df.salesyear.isna().sum()}')
df['salesyear'] = df['salesyear'].fillna('2015')
print(f'num nan sales year after: {df.salesyear.isna().sum()}')

In [ ]:
# car age calculation
df['salesyear'] = df['salesyear'].astype(np.int64)
df['car_age'] = df['salesyear'] - df['year']
pd.unique(df['car_age'])

In [ ]:
# Some of the sales are larger than the year of the car, for these lines we will assume that the age of the car is equal to the year of sale
def age(x):
    if x == -1:
       return  0
    else:
       return x
        
df['car_age'] = df['car_age'].apply(age)
pd.unique(df['car_age'])

In [ ]:
plt.figure(figsize = (5,3))
plt.title('cars age', loc = 'center')
plt.hist(df['car_age'], bins = 35, align = 'mid')
plt.xlabel('car_age', loc = 'center')

### State

In [ ]:
df['state'].unique()

In [ ]:
# Replacing unknown values with NAN values
df['state'] = df['state'].apply(lambda x: np.nan if x[0] == '3' else x)
df['state'].unique()

In [ ]:
# Transfer state names to a full name
df['state'].replace({'Ca': 'California', 'Tx': 'Texas', 'Pa': 'Pennsylvania', 'Mn': 'Minnesota',
    'Az': 'Arizona', 'Wi': 'Wisconsin', 'Tn': 'Tennessee', 'Md': 'Maryland', 'Fl': 'Florida', 'Ne': 'Nebraska',
    'Nj': 'New Jersey', 'Nv': 'Nevada', 'Oh': 'Ohio', 'Mi': 'Michigan', 'Ga': 'Georgia', 'Va': 'Virginia',
    'Sc': 'South Carolina', 'Nc': 'North Carolina', 'In': 'Indiana', 'Il': 'Illinois', 'Co': 'Colorado',
    'Ut': 'Utah', 'Mo': 'Missouri', 'Ny': 'New York', 'Ma': 'Massachusetts', 'Pr': 'Puerto Rico', 'Or': 'Oregon',
    'La': 'Louisiana', 'Wa': 'Washington', 'Hi': 'Hawaii', 'Qc': 'Quebec',  'Ab': 'Alberta', 'On': 'Ontario', 
    'Ok': 'Oklahoma', 'Ms': 'Mississippi', 'Nm': 'New Mexico', 'Al': 'Alabama', 'Ns': 'Nova Scotia'}, inplace = True)

df['state'].unique()

In [ ]:
# distribution of vehicles by company
state = df['state'].value_counts().index
plt.figure(figsize = (20,10))
plt.title('count data by state', loc = 'center')
sb.countplot(data = df, y = 'state', order = state, width = 1.1)
plt.xlabel('number of cars', loc = 'center')

In [ ]:
# Move groups that have less than 200 observations to "other"
print(f"groups that have less than 200 observations before: {df['state'].value_counts()[df['state'].value_counts()<201]}")

df = group_rare_categories(df, 'state', 201)

print(f"groups that have less than 200 observations after: {df['state'].value_counts()[df['state'].value_counts()<201]}")

### Transmission

In [ ]:
df['transmission'].unique()

In [ ]:
# Some columns should be body instead of transmission
df['body'].unique()

In [ ]:
# Here we can see the errors
print(f"Transmission column: {df[df['transmission'] == 'Sedan']['body'].unique()}")
print(f"Body column: {df[df['body'] == 'Navitgation']['transmission'].unique()}")

In [ ]:
# Replace content from transmission to body column and delete these records from transmission
for i, row in df.iterrows():
    if row['transmission'] == 'Sedan':
        df.loc[i, 'body'] = 'Sedan'
        df.loc[i, 'transmission'] = np.nan
print(f'transmission unique: {df.transmission.unique()}')

In [ ]:
cars_transmission = df.groupby('transmission').agg({'transmission' : 'count'})
plt.figure(figsize = (7,5))
plt.title('transmission aut/man', loc = 'center')
plt.pie(cars_transmission['transmission'], labels = cars_transmission['transmission'] )
plt.legend(labels = cars_transmission.index, fontsize = 10, loc='upper left')

### Body

In [ ]:
df['body'].unique()

In [ ]:
# Fix it to have less unique categories
df['body'].replace({'G37 Coupe' : 'Coupe', 'Cts Wagon' : 'Wagon', 'Cts-V Wagon' : 'Wagon',
                    'G37 Convertible' : 'Convertible', 'G Sedan' : 'Sedan', 'G Convertible' : 'Convertible', 'G Coupe': 'Coupe',
                    'Granturismo Convertible' : 'Convertible', 'Ram Van' : 'Van', 'Transit Van': 'Van', 'Q60 Convertible' : 'Convertible',
                    'Q60 Coupe': 'Coupe', 'Tsx Sport Wagon' : 'Sport Wagon', 'Beetle Convertible': 'Convertible',
                    'E-Series Van' : 'Van', 'Elantra Coupe' : 'Coupe', 'Genesis Coupe': 'Coupe', 'Koup' : 'Coupe',
                    'Tsx Sport Wagon' : 'Wagon', 'Cts Coupe' : 'Coupe' , 'Cts-V Coupe' : 'Coupe', 'Promaster Cargo Van' : 'Van' ,
                    'Supercrew' : 'Cab', 'Double Cab' : 'Cab' , 'Access Cab' : 'Cab',
                    'King Cab' : 'Cab', 'Extended Cab' : 'Cab', 'Supercab' : 'Cab', 'Regular Cab' : 'Cab', 'Quad Cab' : 'Cab',
                    'Club Cab' : 'Cab', 'Xtracab' : 'Cab', 'Mega Cab' : 'Cab', 'Cab Plus 4' : 'Cab', 'Cab Plus' : 'Cab',
                     'Crewmax Cab': 'Cab', 'Crew Cab': 'Cab' , 'Regular-Cab': 'Cab' }, inplace = True)
df['body'].unique()

In [ ]:
# distribution of sales by body
body_order = df['body'].value_counts().reset_index()['body']
plt.figure(figsize = (4,5))
sb.catplot(y='body', data=df, kind='count', order = body_order )
plt.title('sales by body', loc = 'center')
plt.xlabel('body', loc = 'center')
plt.show()

### Color

In [ ]:
df['color'].unique()

In [ ]:
# Transferring irrelevant data to nan
def delete_non_state(x):
    del_list = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '—']
    for i in str(x):
        if str(i) in del_list:
            return np.nan
        else:
            return x
df['color'] = df['color'].apply(lambda x: delete_non_state(x))
df['color'].unique()

In [ ]:
# distribution of sales by color
color_order = df['color'].value_counts().reset_index()['color']
plt.figure(figsize = (5,4))
sb.catplot(y='color', data=df, kind='count', order = color_order)
plt.title('sales by color', loc = 'center')
plt.xlabel('sales', loc = 'center')

In [ ]:
# Move groups that have less than 250 observations to "other"
print(f"groups that have less than 250 observations before: {df['color'].value_counts()[df['color'].value_counts()<251]}")

df = group_rare_categories(df, 'color', 251)

print(f"groups that have less than 250 observations after: {df['color'].value_counts()[df['color'].value_counts()<251]}")

### Interior

In [ ]:
df['interior'].unique()

In [ ]:
# Replacing values "-" to nan
df['interior'].replace({'—': np.nan}, inplace = True)

In [ ]:
# Move groups that have less than 200 observations to "other"
print(f"groups that have less than 200 observations before: {df['interior'].value_counts()[df['interior'].value_counts()<201]}")

df = group_rare_categories(df, 'interior', 201)

print(f"groups that have less than 200 observations after: {df['interior'].value_counts()[df['interior'].value_counts()<201]}")

### Condition

In [ ]:
np.sort(df['condition'].unique())

In [ ]:
# data distribution by condition
plt.figure(figsize = (5,3))
plt.title('count cars by condition', loc = 'center')
plt.hist(df['condition'], bins = 25, align = 'mid')
plt.xlabel('condition', loc = 'center')

In [ ]:
# It looks like Condition was rated 1-5 or 10-50. We will transfer it to 10-50 scale.
# Data transfer to 1-50 scale
df['condition'] = df['condition'].apply(lambda x: x*10 if x < 10 else x)
np.sort(df['condition'].unique())

### Odometer

In [ ]:
# Checking the distribution of values and outliers
fig, place = plt.subplots(nrows=2,ncols=1,figsize=(13,5),sharex=True)
plt.suptitle('odometer distribution')
plt.xlabel('odometer', loc = 'center')

place[0].boxplot(df[df['odometer'].notna()]['odometer'], vert = False)
place[1].hist(df[df['odometer'].notna()]['odometer'],bins=60,alpha=0.5,density=True)

plt.tight_layout()
plt.show()

In [ ]:
# check abnormal values, how many rows there is a mileage equal to 1 and the age of the vehicle is over a year
df['odometer'] = pd.to_numeric(df['odometer'])
print(f"mileage equal to 1: {df[(df['odometer']<2)]['odometer'].count()}")
print(f"mileage equal to 1 and the age of the vehicle is over a year: {df[(df['odometer']<2) & (df['car_age']>0)]['odometer'].count()}")
print(f"mileage equal to 1 and the age of the vehicle is less than a year: {df[(df['odometer']<2) & (df['car_age']==0)]['odometer'].count()}")
# check rows with mileage over 900000 
print(f"mileage over 900000: {df[df['odometer']>900001]['odometer'].count()}")
# transfer rows to nan
df['odometer'] = df.apply(lambda x: np.nan if (x['odometer'] < 2 and x['car_age'] > 0) or (x['odometer'] > 990000) else x['odometer'], axis=1)

In [ ]:
print(f"mileage equal to 1: {df[(df['odometer']<2)]['odometer'].count()}")
print(f"mileage equal to 1 and the age of the vehicle is over a year: {df[(df['odometer']<2) & (df['car_age']>0)]['odometer'].count()}")
print(f"mileage equal to 1 and the age of the vehicle is less than a year: {df[(df['odometer']<2) & (df['car_age']==0)]['odometer'].count()}")
print(f"mileage over 990000: {df[df['odometer']>990000]['odometer'].count()}")

### Saleprice

In [ ]:
# Checking the distribution of values and outliers
fig, place = plt.subplots(nrows=2,ncols=1,figsize=(13,5),sharex=True)

plt.suptitle('Selling Price distribution')
plt.xlabel('price', loc = 'center')

place[0].boxplot(df[df['sellingprice'].notna()]['sellingprice'],vert=False)
place[1].hist(df[df['sellingprice'].notna()]['sellingprice'],bins=60,alpha=0.5,density=True)

plt.tight_layout()
plt.show()

In [ ]:
# Checking the distribution of values ​​and outliers by year 
plt.figure(figsize = (15,5))
plt.title('outleyers by year', loc = 'center')
sb.boxplot(x='year', y='sellingprice', data=df, palette='rainbow')
plt.xlabel('year', loc = 'center')

In [ ]:
df['price_res']=df['mmr']-df['sellingprice']
df['price_res'].describe().round(2)

In [ ]:
# sales price under 150$
print(f"sales price under 150$: {df[df['sellingprice']<150]['sellingprice'].count()}")
# sales price residuals under or upper 25000$
print(f"sales price residuals under or upper 25000$: {df[(df['price_res']>25000) | (df['price_res']<-25000)]['price_res'].count()}")
# delete rows 
df = df[(df['sellingprice']>150) & (df['price_res']<25000) & (df['price_res']>-25000)]

## Data completion/deletion

In [ ]:
df.info()

In [ ]:
# For a sales predictor, some columns are irrelevant
df.drop(columns = ['year', 'vin', 'saledate', 'salesyear', 'price_res', 'seller'], inplace = True)

In [ ]:
# missing values in database
df.isna().sum()

In [ ]:
# Make is the most important data, therefore if the row is nan - we will delete rows
df.dropna(axis = 0, subset = ['make', 'model'],  inplace = True)

In [ ]:
# Also delete lines with a missing combination of model and trim
df.dropna(axis = 0, subset = ['model', 'trim'], how = 'all',  inplace = True)

In [ ]:
# To compare at the end
print(f'shape before filling: {df.shape}')
print(f'nun of nan before filling: {df.isna().sum()}')

In [ ]:
# Complete color column
# If color is Nan, fill it based on Make-Model most common color.
color_mode = df.groupby(by=['make', 'model'])['color'].agg(pd.Series.mode).explode().reset_index()
color_mode = color_mode.groupby(by=['make', 'model']).first().reset_index()
df = pd.merge(left=df, right=color_mode, on=['make', 'model'], how='left')
df['color_x'] = df['color_x'].fillna(df['color_y'])
df.drop('color_y', axis=1, inplace=True)
df.rename(columns={'color_x': 'color'}, inplace=True)

In [ ]:
# Same for body column
body_mode = df.groupby(by = ['make', 'model'])['body'].agg(pd.Series.mode).explode().reset_index()
body_mode = body_mode.groupby(by = ['make', 'model']).first().reset_index()
df = pd.merge(left = df, right = body_mode, on = ['make', 'model'], how = 'left')
df['body_x'] = df['body_x'].fillna(df['body_y'])
df.drop('body_y', axis = 1, inplace = True)
df.rename(columns = {'body_x' : 'body'}, inplace = True)

In [ ]:
# Same for transmission
transmission_mode = df.groupby(by = ['make', 'model'])['transmission'].agg(pd.Series.mode).explode().reset_index()
transmission_mode = transmission_mode.groupby(by = ['make', 'model']).first().reset_index()
df = pd.merge(left = df, right = transmission_mode, on = ['make', 'model'], how = 'left')
df['transmission_x'] = df['transmission_x'].fillna(df['transmission_y'])
df.drop('transmission_y', axis = 1, inplace = True)
df.rename(columns = {'transmission_x' : 'transmission'}, inplace = True)

In [ ]:
# Same for interior
interior_mode = df.groupby(by = ['make', 'model'])['interior'].agg(pd.Series.mode).explode().reset_index()
interior_mode = interior_mode.groupby(by = ['make', 'model']).first().reset_index()
df = pd.merge(left = df, right = interior_mode, on = ['make', 'model'], how = 'left')
df['interior_x'] = df['interior_x'].fillna(df['interior_y'])
df.drop('interior_y', axis = 1, inplace = True)
df.rename(columns = {'interior_x' : 'interior'}, inplace = True)

In [ ]:
# Same for trim
trim_mode = df.groupby(by = ['make', 'model'])['trim'].agg(pd.Series.mode).explode().reset_index()
trim_mode = trim_mode.groupby(by = ['make', 'model']).first().reset_index()
df = pd.merge(left = df, right = trim_mode, on = ['make', 'model'], how = 'left')
df['trim_x'] = df['trim_x'].fillna(df['trim_y'])
df.drop('trim_y', axis = 1, inplace = True)
df.rename(columns = {'trim_x' : 'trim'}, inplace = True)

In [ ]:
# Could have been done with a function, I didn't do it for clarity
# This function fills NaN values in a specific column with the most frequent value within each make-model pair
def fill_mode(df, column_name):
    mode_df = df.groupby(by=['make', 'model'])[column_name].agg(pd.Series.mode).explode().reset_index()
    mode_df = mode_df.groupby(by=['make', 'model']).first().reset_index()
    
    df = pd.merge(left=df, right=mode_df, on=['make', 'model'], how='left')
    df[f'{column_name}_x'] = df[f'{column_name}_x'].fillna(df[f'{column_name}_y'])
    
    df.drop(f'{column_name}_y', axis=1, inplace=True)
    df.rename(columns={f'{column_name}_x': column_name}, inplace=True)
    
    return df

In [ ]:
# For condition we do median instead of mode
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

condition_mean = df.groupby(by = ['car_age', 'odometer'], dropna =True)['condition'].agg(pd.Series.median).reset_index()
condition_mean['condition'] = condition_mean['condition'].round(0)
df = pd.merge(left=df, right=condition_mean, on=['car_age', 'odometer'], how='left')
df['condition_x'] = df['condition_x'].fillna(df['condition_y'])
df.drop('condition_y', axis=1, inplace=True)
df.rename(columns={'condition_x': 'condition'}, inplace=True)

In [ ]:
# Compare with previous print
print(f'shape after filling: {df.shape}')
print(f'nun of nan after filling: {df.isna().sum()}')

In [ ]:
# Delete the rest of nans that were not completed 
df.dropna(how = 'any', inplace = True)
print(f'nun of nan after filling: {df.shape}')
print(f'shape after filling: {df.isna().sum()}')

In [ ]:
# Fine tuning
df.rename(columns = {'car_age' : 'age'} ,inplace = True)

In [ ]:
# Duplication check
df.duplicated().sum()

In [ ]:
df.describe().round(1).T

In [ ]:
# Save dataframe after FE
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
save_path = os.path.join(notebook_dir, '../data/processed/car_prices_after_FE_3.csv')

df.to_csv(save_path, index=False)